In [ ]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [1]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, RNN, SimpleRNNCell, Embedding, Dropout

In [2]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [3]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"(spoilers)Wow, this is a bad one. I did a double take when watching an old Star Trek episode the other day-it was the one where everyone gets infected with that space sickness and then go a bit nuts-and there was Stewart Moss, a.k.a the unlikable 'hero' of It Lives by Night! He played the first crewmember infected, who dies from terminal depression. All I could think was that he'd watched his own movie too many times, that's what caused the depression. This movie is full of truly unlikable people. There is no redeeming character in the film, not one. It's very hard to feel bad about Dr. Beck's turning into a bat(or whatever he actually turned into), because you just don't like him. And you don't like his shrill, bony wife, or the nasty sleazy Sgt. Ward, or Dr. Mustache Love...So why would you invest any time or energy in this movie? Where there is no empathy with the characters, there is no reason to 

In [4]:
model = Sequential()

In [5]:
model.add(Input(shape=(1,), dtype="string"))

In [6]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

In [7]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [8]:
model.add(Embedding(max_tokens + 1, 128))

rnn = RNN(SimpleRNNCell(64) , return_sequences=False,return_state=False)
model.add(rnn)
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [9]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 63s 78ms/step - loss: 0.6938 - accuracy: 0.5165
Epoch 2/10
782/782 [==============================] - 60s 77ms/step - loss: 0.6854 - accuracy: 0.5494
Epoch 3/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6527 - accuracy: 0.6087
Epoch 4/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6632 - accuracy: 0.5975
Epoch 5/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6658 - accuracy: 0.5897
Epoch 6/10
782/782 [==============================] - 60s 76ms/step - loss: 0.6394 - accuracy: 0.6265
Epoch 7/10
782/782 [==============================] - 59s 75ms/step - loss: 0.6597 - accuracy: 0.5989
Epoch 8/10
782/782 [==============================] - 60s 77ms/step - loss: 0.6384 - accuracy: 0.6301
Epoch 9/10
782/782 [==============================] - 61s 78ms/step - loss: 0.6140 - accuracy: 0.6589
Epoch 10/10
782/782 [==============================] - 60s 77ms/step - loss: 0.646

In [11]:
model.evaluate(test_data)

782/782 [==============================] - 14s 18ms/step - loss: 0.7134 - accuracy: 0.5574


[0.7133914828300476, 0.5574399828910828]

In [12]:
text = "I loved the movie !"

In [13]:
model.predict([text])

array([[0.58057064]], dtype=float32)